In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/fishnet-ai/foid_images_v100/images | wc -l


In [ ]:
!ls /kaggle/input/fishnet-ai/foid_labels_v100

In [ ]:
!cat /kaggle/input/fishnet-ai/foid_labels_v100/README*

In [ ]:
df = pd.read_csv('/kaggle/input/fishnet-ai/foid_labels_v100/foid_labels_v100.csv')

In [ ]:
df

In [ ]:
df['img_id'][0]

In [ ]:
!ls /kaggle/input/fishnet-ai/foid_images_v100/images | grep da7c5ef0-db28-11ea-b693-9b3523cc10a5

In [ ]:
df['fao_mfa'].value_counts()

In [ ]:
df.columns

In [ ]:
df = df.drop(['cam_id', 'seq_id', 'frame_id', 'bbox_id', 'label_l2', 'fao_mfa'],axis=1)

In [ ]:
df 

In [ ]:
image_path = '/kaggle/input/fishnet-ai/foid_images_v100/images/'

In [ ]:
from PIL import Image

In [ ]:
test_img = Image.open(f'{image_path}/da7c5ef0-db28-11ea-b693-9b3523cc10a5.jpg')
test_img.size

In [ ]:
first_image_bboxes = df[df['img_id']=='da7c5ef0-db28-11ea-b693-9b3523cc10a5']
first_image_bboxes

In [ ]:
from PIL import ImageDraw 
  

# create rectangle image 
img1 = ImageDraw.Draw(test_img)
img1.rectangle([626,-1,754,88], outline ="red") 
img1.rectangle([544,15,710,203], outline ="red") 
img1.rectangle([629,210,735,562], outline ="red") 
img1.rectangle([885,826,962,875], outline ="red") 
img1.rectangle([615,550,883,907], outline ="red") 



In [ ]:
img1._image

In [ ]:
df_train = df[df['train']]
df_val = df[df['val']]
df_test =  df[df['test']]
len(df_train),len(df_test),len(df_val)

In [ ]:
df_train.columns

In [ ]:
map_ids = {name: num for num, name in enumerate(set(df['label_l1']))}

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, images, labels, transform=None, target_transform=None):
        super().__init__()
        self.labels = labels
        self.images = images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [ ]:
import cv2


In [ ]:
print(next(df.iterrows()))

In [ ]:
from tqdm import tqdm

In [ ]:
from shutil import copy

In [ ]:

# Dictionary to convert label to class_id (you can modify it according to your labels)

# Folder to save annotations
os.makedirs('annotations', exist_ok=True)
os.makedirs('annotations/images', exist_ok=True)
os.makedirs('annotations/images/train', exist_ok=True)
os.makedirs('annotations/images/val', exist_ok=True)
os.makedirs('annotations/images/test', exist_ok=True)
os.makedirs('annotations/labels', exist_ok=True)
os.makedirs('annotations/labels/train', exist_ok=True)
os.makedirs('annotations/labels/val', exist_ok=True)
os.makedirs('annotations/labels/test', exist_ok=True)

# Loop over the dataframe to generate annotations
for idx, row in tqdm(df_train.iterrows(),total=len(df_train)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    # Open image to get width and height
    # Assuming you use OpenCV (cv2) to get image size
    img = cv2.imread(img_path)
    h, w, _ = img.shape  # height, width, channels

    # Convert coordinates to YOLO format
    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    # Get class_id from label
    class_id = map_ids[row['label_l1']]
    
    # Create annotation file path
    txt_file_path = os.path.join('annotations/labels/train', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/train', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    # Write to annotation file
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)



In [ ]:
for idx, row in tqdm(df_test.iterrows(),total=len(df_test)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    # Open image to get width and height
    # Assuming you use OpenCV (cv2) to get image size
    img = cv2.imread(img_path)
    h, w, _ = img.shape  # height, width, channels

    # Convert coordinates to YOLO format
    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    # Get class_id from label
    class_id = map_ids[row['label_l1']]
    
    # Create annotation file path
    txt_file_path = os.path.join('annotations/labels/test', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/test', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    # Write to annotation file
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)


In [ ]:
for idx, row in tqdm(df_val.iterrows(),total=len(df_val)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    # Open image to get width and height
    # Assuming you use OpenCV (cv2) to get image size
    img = cv2.imread(img_path)
    h, w, _ = img.shape  # height, width, channels

    # Convert coordinates to YOLO format
    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    # Get class_id from label
    class_id = map_ids[row['label_l1']]
    
    # Create annotation file path
    txt_file_path = os.path.join('annotations/labels/val', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/val', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    # Write to annotation file
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)


In [ ]:
import json
with open(txt_file_path, 'w') as f:
    json.dump(map_ids,f)

In [ ]:
map_ids